In [10]:
# given target and source 
# move source pc
import ICP_stuff
from tqdm import tqdm
from scipy.spatial.distance import cdist
import numpy as np

def icp_corres(source, target):
    dist=cdist(source, target)
    idxs= np.argmin(dist, axis= 1)
    cost= np.linalg.norm(target[idxs],source).mean()
    return cost, idxs

def icp_loop_runner(source_pc, target_pc, init_T, max_iters=100 ):
    R, p= ICP_stuff.get_R_and_P(init_T)
    prev_dist= np.inf
    print(R,p)
    for itr in tqdm(range(max_iters)):
        moved_src_pc= ICP_stuff.rotate_pc(R, source_pc) + p
        dist, idxs = icp_corres(moved_src_pc,target_pc)
        src_centroid = ICP_stuff.get_pc_centroid(moved_src_pc)
        tar_centroid = ICP_stuff.get_pc_centroid(target_pc[idxs])
        centred_src_pc= moved_src_pc - src_centroid
        centred_tar_pc= target_pc[idxs] - tar_centroid
        import ipdb; ipdb.set_trace()
        Q= centred_src_pc.T @ centred_tar_pc
        # import ipdb; ipdb.set_trace()        
        U, S, Vt = np.linalg.svd(Q, full_matrices=True)
        diag_mat= np.diag([1,1,np.linalg.det(U@Vt)])
        R= U@diag_mat@Vt
        p= tar_centroid - R @ src_centroid
        
        if np.abs(dist-prev_dist) < 1e-5:
            return dist, ICP_stuff.get_pose(R,p)
            break
        prev_dist= dist
    return prev_dist, ICP_stuff.get_pose(R,p)

In [33]:
import transforms3d as t3d
number_of_yaw_splits=4

yaw_split= (2*np.pi)/number_of_yaw_splits
max_iters= 100

euclidean_dist= []
best_pose=[]

for ii in range(number_of_yaw_splits):
    yaw_val= yaw_split*ii
    print('\n\n###########################\n\n')
    print("YAW_VALUE :: ", yaw_val)
    init_R= t3d.euler.euler2mat(0,0, yaw_val)
    init_p= np.zeros(3)
    init_T= ICP_stuff.get_pose(init_R, init_p)
    print("Initial Pose:\n ",init_T)
    # target_pc= target_pc[::5]
    # source_pc= source_pc[::5]
    dist, pose= icp_loop_runner_v2(source_pc, target_pc, init_T, max_iters=100 )
    best_pose.append(pose)
    euclidean_dist.append(dist)



###########################


YAW_VALUE ::  0.0
Initial Pose:
  [[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [-0.  0.  1.  0.]
 [ 0.  0.  0.  1.]]
[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [-0.  0.  1.]] [0. 0. 0.]


100%|██████████| 100/100 [00:08<00:00, 12.12it/s]




###########################


YAW_VALUE ::  1.5707963267948966
Initial Pose:
  [[ 6.123234e-17 -1.000000e+00  0.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00  0.000000e+00]
 [-0.000000e+00  0.000000e+00  1.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]
[[ 6.123234e-17 -1.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00]
 [-0.000000e+00  0.000000e+00  1.000000e+00]] [0. 0. 0.]


100%|██████████| 100/100 [00:09<00:00, 10.40it/s]




###########################


YAW_VALUE ::  3.141592653589793
Initial Pose:
  [[-1.0000000e+00 -1.2246468e-16  0.0000000e+00  0.0000000e+00]
 [ 1.2246468e-16 -1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-0.0000000e+00  0.0000000e+00  1.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[[-1.0000000e+00 -1.2246468e-16  0.0000000e+00]
 [ 1.2246468e-16 -1.0000000e+00  0.0000000e+00]
 [-0.0000000e+00  0.0000000e+00  1.0000000e+00]] [0. 0. 0.]


100%|██████████| 100/100 [00:10<00:00,  9.81it/s]




###########################


YAW_VALUE ::  4.71238898038469
Initial Pose:
  [[-1.8369702e-16  1.0000000e+00 -0.0000000e+00  0.0000000e+00]
 [-1.0000000e+00 -1.8369702e-16  0.0000000e+00  0.0000000e+00]
 [-0.0000000e+00  0.0000000e+00  1.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00]]
[[-1.8369702e-16  1.0000000e+00 -0.0000000e+00]
 [-1.0000000e+00 -1.8369702e-16  0.0000000e+00]
 [-0.0000000e+00  0.0000000e+00  1.0000000e+00]] [0. 0. 0.]


100%|██████████| 100/100 [00:09<00:00, 11.02it/s]


In [4]:
import numpy as np
from utils import read_canonical_model, load_pc, visualize_icp_result

obj_name = 'drill' # drill or liq_container
num_pc = 4 # number of point clouds

source_pc = read_canonical_model(obj_name)

for i in range(num_pc):
    target_pc = load_pc(obj_name, i)
    break
    # ICP(target_pc,source_pc, viz=True, number_of_yaw_splits=36  )
    # break

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
def ICP(source_pc, target_pc,init_T):
    max_iters= 100
    filter_dist_threshold= 0.5

    sampled_target_pc= target_pc[::]
    sampled_source_pc= source_pc[::]
    euclidean_dist= []
    best_pose=[]


    # print("Initial Pose:\n ",init_T)
    init_R, init_p= get_R_and_P(init_T)  

    moved_target_pc= rotate_pc(init_R, sampled_target_pc)+init_p
    prev_euclidean_dist= 0
    accumulated_pose = init_T
    dist, corres_idxs = get_corres_points(moved_target_pc,sampled_source_pc)
    # print(dist)
    dist_between_corresponding_pc_pts= np.linalg.norm(sampled_source_pc-moved_target_pc[corres_idxs],axis=1)
    
    filtered_idx = np.where(dist_between_corresponding_pc_pts < filter_dist_threshold)
    
    sampled_source_pc=sampled_source_pc[filtered_idx]
    # moved_target_pc=moved_target_pc[corres_idxs][filtered_idx]
    moved_target_pc = moved_target_pc[corres_idxs][filtered_idx]
    for itr in (range(max_iters)):
        dist, corres_idxs = get_corres_points(moved_target_pc,sampled_source_pc)

        #Kabasch stuff
        src_centroid = get_pc_centroid(sampled_source_pc)
        tar_centroid = get_pc_centroid(moved_target_pc)
    
        centered_sampled_source_pc= sampled_source_pc - src_centroid
        centered_moved_target_pc= moved_target_pc- tar_centroid
    
        Q= np.matmul(centered_sampled_source_pc.transpose(), centered_moved_target_pc[corres_idxs])
        U, S, Vt = np.linalg.svd(Q, full_matrices=True)
        R= U@np.diag([1,1,np.linalg.det(U@Vt)])@Vt
        
        p= src_centroid - rotate_pc(R, tar_centroid)

        pred_pose= get_pose(R,p)
        accumulated_pose= pred_pose@accumulated_pose

        moved_target_pc = rotate_pc( R, moved_target_pc) + p
        if np.abs(dist-prev_euclidean_dist) < 1e-10:
            best_dist= dist
            best_pose = accumulated_pose
            return best_pose
            break
        prev_euclidean_dist = dist
        # print(dist)
    # print(accumulated_pose, best_dist)
    euclidean_dist.append(best_dist)
    best_pose.append(accumulated_pose)
    return accumulated_pose

In [16]:
def icp_loop_runner_v2(source_pc, target_pc, init_T, max_iters=100 ):
    R, p= ICP_stuff.get_R_and_P(init_T)
    prev_dist= np.inf
    print(R,p)
    accumulated_pose= np.identity(4)
    for itr in tqdm(range(max_iters)):
        moved_src_pc= ICP_stuff.rotate_pc(R, source_pc) + p
        dist, idxs = get_corres_points_v2(moved_src_pc,target_pc)
        # Kabasch stuff
        src_centroid = ICP_stuff.get_pc_centroid(moved_src_pc)
        tar_centroid = ICP_stuff.get_pc_centroid(target_pc[idxs])
        centred_src_pc= moved_src_pc - src_centroid
        centred_tar_pc= target_pc[idxs] - tar_centroid
        # import ipdb; ipdb.set_trace()
        Q= centred_src_pc.T @ centred_tar_pc
        # import ipdb; ipdb.set_trace()        
        U, S, Vt = np.linalg.svd(Q, full_matrices=True)
        diag_mat= np.diag([1,1,np.linalg.det(U@Vt)])
        R= U@diag_mat@Vt
        p= tar_centroid - R @ src_centroid
        accumulated_pose= accumulated_pose@ICP_stuff.get_pose(R,p) 
        if np.abs(dist-prev_dist) < 1e-5:
            return dist, accumulated_pose
            break
        prev_dist= dist
        
    return prev_dist, ICP_stuff.get_pose(R,p)

In [32]:
from sklearn.neighbors import NearestNeighbors
import transforms3d as t3d
def get_corres_points_v2(source_pc, target_pc):
    kn_obj=NearestNeighbors(n_neighbors=1).fit(target_pc)
    dist, idx= kn_obj.kneighbors(source_pc)
    idxs= idx.flatten()
    cost= (np.linalg.norm(target_pc[idxs]-source_pc, axis=1)**2).mean()
    return cost, idxs